<a href="https://colab.research.google.com/github/JericCantos/SentencePolarity/blob/main/notebooks/Sentence_Polarity_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Using the [Sentence Polarity dataset](https://www.kaggle.com/datasets/nltkdata/sentence-polarity), train and evaluate different sentiment classsifier models that can predict whether given review is leaning **positive** or **negative**

The project goes through:
1. Data Preprocessing and Cleaning
2. Feature Extraction
3. Model Trainning and Evaluation with Cross-Validation
4. Hyperparameter Tuning

# Setup

## Import Libraries

In [5]:
import numpy as np
import pandas as pd
import random
import nltk
import re

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

# Downloading necessary NLTK resources
nltk.download('stopwords')  # List of common stop words in English
nltk.download('punkt')  # Pre-trained tokenizer models
nltk.download('wordnet')  # WordNet lemmatizer dataset

# Libraries for text feature extraction and model training
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# Libraries for model evaluation
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import KFold, cross_val_score

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load Dataset

### Kaggle

Ensure that you create a KAGGLE_USERNAME and KAGGLE_KEY secret. You can get a username and key by generating an API Token from Kaggle (Settings -> API).

In [2]:
import os
import json
from google.colab import userdata

KAGGLE_USERNAME = userdata.get('KAGGLE_USERNAME')
KAGGLE_KEY = userdata.get('KAGGLE_KEY')

# Create kaggle.json from your secrets
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump({
        "username": KAGGLE_USERNAME,
        "key": KAGGLE_KEY
    }, f)

!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d nltkdata/sentence-polarity

Dataset URL: https://www.kaggle.com/datasets/nltkdata/sentence-polarity
License(s): other
  0% 0.00/488k [00:00<?, ?B/s]
100% 488k/488k [00:00<00:00, 617MB/s]


In [4]:
!unzip sentence-polarity.zip

Archive:  sentence-polarity.zip
  inflating: sentence_polarity/README.txt  
  inflating: sentence_polarity/rt-polarity.neg  
  inflating: sentence_polarity/rt-polarity.pos  


### Read Datafiles

In [9]:
# Read the positive and negative sentiment files
df_sent_pos = pd.read_csv('sentence_polarity/rt-polarity.pos',
                          sep='\t', header=None)
  # Positive sentiment sentences
df_sent_neg = pd.read_csv('sentence_polarity/rt-polarity.neg', sep='\t', header=None)
  # Negative sentiment sentences

print(df_sent_pos.shape)
print(df_sent_neg.shape)
df_sent_pos.head()

(5331, 1)
(5331, 1)


,0
0,the rock is destined to be the 21st century's ...
1,"the gorgeously elaborate continuation of "" the..."
2,effective but too-tepid biopic
3,if you sometimes like to go to the movies to h...
4,"emerges as something rare , an issue movie tha..."


In [10]:
# Rename the column to 'sentence'
df_sent_pos.rename(columns={0: "sentence"}, inplace=True)
df_sent_neg.rename(columns={0: "sentence"}, inplace=True)

# Data Preprocessing

In [11]:
# Define the preprocessing function
def preprocess_text(sentences):
    # Convert all tokens to lowercase
    sentences = [sentence.lower() for sentence in sentences]

    # Remove punctuation using regex
    sentences = [re.sub(r"[^\w\s]", "", sentence) for sentence in sentences]

    # Remove extra whitespace between words
    sentences = [" ".join(sentence.split()) for sentence in sentences]

    # Tokenize sentences into words
    sentences = [word_tokenize(sentence) for sentence in sentences]

    # Remove stop words
    stop_words = set(stopwords.words('english'))  # Load English stop words
    filtered_sentences = []
    for sentence in sentences:
        filtered_sentence = [word for word in sentence if word not in stop_words]
        filtered_sentences.append(filtered_sentence)

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []
    for sentence in filtered_sentences:
        lemmatized_sentence = [lemmatizer.lemmatize(word) for word in sentence]
        lemmatized_sentences.append(lemmatized_sentence)

    return [' '.join(sentence) for sentence in lemmatized_sentences]

In [12]:
# Preprocess the sentences
pos_preprocessed_sentences = preprocess_text(df_sent_pos['sentence'])
neg_preprocessed_sentences = preprocess_text(df_sent_neg['sentence'])

# Print the first preprocessed negative sentence
print(neg_preprocessed_sentences[0])

simplistic silly tedious


In [14]:
# Combine preprocessed positive and negative sentences
sentences = pos_preprocessed_sentences + neg_preprocessed_sentences

In [15]:
# Create a list for all labels
polarities = []
polarities.extend([1] * len(df_sent_pos))  # Label positive sentences as 1
polarities.extend([0] * len(df_sent_neg))  # Label negative sentences as 0

In [18]:
# Combine sentences and labels into a single list
combined = list(zip(sentences, polarities))

# Shuffle the combined list
random.shuffle(combined)

# Split the shuffled list back into sentences and labels
sentences[:], polarities[:] = zip(*combined)

# Train-Test Split

In [19]:
# Define train-test split ratio
train_test_ratio = 0.8

# Calculate the size of the training set
train_set_size = int(train_test_ratio * len(sentences))

# Split data into training and test sets
X_train, X_test = sentences[:train_set_size], sentences[train_set_size:]
y_train, y_test = polarities[:train_set_size], polarities[train_set_size:]

# Print sizes of training and test sets
print("Size of training set:", len(X_train))
print("Size of test set:", len(X_test))

Size of training set: 8529
Size of test set: 2133


# Feature Extraction

Default TfidVectorizer includes:

- Only unigrams (individual terms/tokens) as features.
- A maximum document frequency of 1.0 (no terms are excluded based on frequency).
- Normalization applied to the resulting feature vectors.

In [20]:
# Define the vectorizer with default parameters
tfidf_vectorizer = TfidfVectorizer()

# Transform the training data into a TF-IDF matrix
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Check the number of samples and features
num_samples, num_features = X_train_tfidf.shape
print("#Samples: {}, #Features: {}".format(num_samples, num_features))

#Samples: 8529, #Features: 16528


# Logistic Regression

In [21]:
# Train
model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)

LogisticRegression()

In [22]:
# Test
X_test_tfidf = tfidf_vectorizer.transform(X_test)
y_pred_lr = model_lr.predict(X_test_tfidf)

In [36]:
print (confusion_matrix(y_test, y_pred_lr))
print (classification_report(y_test, y_pred_lr))
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.3f}")

[[807 252]
 [251 823]]
              precision    recall  f1-score   support

           0       0.76      0.76      0.76      1059
           1       0.77      0.77      0.77      1074

    accuracy                           0.76      2133
   macro avg       0.76      0.76      0.76      2133
weighted avg       0.76      0.76      0.76      2133

Accuracy: 0.764


# Cross-Validation

In [30]:
# Perform 10-fold cross-validation on the training data
f1_scores_list = cross_val_score(
    LogisticRegression(),
    X_train_tfidf,
    y_train,
    cv=10,
    scoring="f1"
)

# Display the F1 scores for each fold
print(f"F1 Scores for each fold: {f1_scores_list}")

# Calculate and display the mean and standard deviation of the F1 scores
print("F1 Score (Mean/Average): {:.3f}".format(f1_scores_list.mean()))
print("F1 Score (Standard Deviation): {:.3f}".format(f1_scores_list.std()))

F1 Scores for each fold: [0.73831776 0.75732708 0.76023392 0.73164862 0.76122931 0.71046229
 0.75771971 0.75592417 0.735363   0.75294118]
F1 Score (Mean/Average): 0.746
F1 Score (Standard Deviation): 0.016


# Hyperparameter Tuning

In [32]:
# Initialize placeholders to store the best configuration
best_score = -1.0
best_classifier = None
best_ngram_size = -1

# Define the hyperparameters to test
classifiers = [LinearSVC(), LogisticRegression(solver="sag")]
            # SAG - stochastic average gradient, useful for large datasets
ngram_sizes = [1, 2, 3, 4]
            # ngram - number of consecutive words to be taken as features

# Loop through all combinations of classifiers and n-gram sizes
for classifier in classifiers:
    for n in ngram_sizes:
        # Define the vectorizer with the current n-gram size
        vectorizer = TfidfVectorizer(ngram_range=(1, n))
        X_train_tfidf = vectorizer.fit_transform(X_train)  # Transform training data

        # Perform 10-fold cross-validation
        f1_scores = cross_val_score(classifier, X_train_tfidf, y_train, cv=10, scoring='f1')
        avg_f1_score = f1_scores.mean()  # Calculate average F1-score

        # Print the result for this combination
        print(f"Classifier: {type(classifier).__name__}, n-gram size: {n} => F1-score: {avg_f1_score:.3f}")

        # Save the best configuration
        if avg_f1_score > best_score:
            best_score = avg_f1_score
            best_classifier = classifier
            best_ngram_size = n

# Print the best configuration
print("\nBest Configuration:")
print(f"Classifier: {type(best_classifier).__name__}, Best n-gram size: {best_ngram_size}, F1-score: {best_score:.3f}")

Classifier: LinearSVC, n-gram size: 1 => F1-score: 0.748
Classifier: LinearSVC, n-gram size: 2 => F1-score: 0.756
Classifier: LinearSVC, n-gram size: 3 => F1-score: 0.757
Classifier: LinearSVC, n-gram size: 4 => F1-score: 0.753
Classifier: LogisticRegression, n-gram size: 1 => F1-score: 0.746
Classifier: LogisticRegression, n-gram size: 2 => F1-score: 0.745
Classifier: LogisticRegression, n-gram size: 3 => F1-score: 0.738
Classifier: LogisticRegression, n-gram size: 4 => F1-score: 0.736

Best Configuration:
Classifier: LinearSVC, Best n-gram size: 3, F1-score: 0.757


# Best Model Evaluation

Based on hyperparameter tuning, the best model would be LinearSVC with an n-gram size of 3

In [38]:
# Use the best configuration to train the final model
final_vectorizer = TfidfVectorizer(ngram_range=(1, best_ngram_size))
X_train_tfidf = final_vectorizer.fit_transform(X_train)
X_test_tfidf = final_vectorizer.transform(X_test)

best_classifier.fit(X_train_tfidf, y_train)
y_pred = best_classifier.predict(X_test_tfidf)

# Evaluate and display results
print("\nFinal Model Results:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")


Final Model Results:
[[790 269]
 [236 838]]
              precision    recall  f1-score   support

           0       0.77      0.75      0.76      1059
           1       0.76      0.78      0.77      1074

    accuracy                           0.76      2133
   macro avg       0.76      0.76      0.76      2133
weighted avg       0.76      0.76      0.76      2133

Accuracy: 0.763


The tuned model "guesses positive" a little more than the baseline linear regression model, but performance seems very comparable overall.